# Pruebas para el tp 2 de NLP

Fuentes de conocimiento:
- Documentos de texto
- Datos numéricos en formato tabular
    - la recuperación de los datos tabulares, debe ser a través de una query dinámica (Por ejemplo con SQL, o filtros de Pandas), y no se debe incorporar tablas completas a la ventana de contexto. Se deben entonces buscar solamente los elementos relevantes de la tabla referidos al prompt.
- Bases de datos de grafos
    - Para crear el contenido de la base de datos de grafos, se debe extraer el
    conocimiento desde una fuente de texto o tabular, y generar luego las tríadas para insertar en la base de datos.
    -  La query a la base de datos de grafos, debe ser dinámica, recuperando solamente elementos relacionados con el prompt. No está permitido traer siempre todo el contenido de la base de datos e incorporarlo como contexto en la aumentación, ya que eso resulta ineficiente y podría consumir muchos tokens. La query puede hacerse con SPARQL, Cypher o similar.
- Pueden añadirse fuentes de información a través de API's públicas (por ejemplo buscar en internet con serper de google search, serpAPI, etc)

Requisitos:
- Conjunto de datos de la menos 100 páginas de texto y un mínimo de 3 documentos.
- Verificar calidad de chunks y limpiar texto extraido si se considera.
- Modelos de embeddings a elección
- LLM a elección.
- Dos versiones del clasificador:
    - Basado en LLM (unidad 6)
    - Basado en un modelo entrenado con ejemplos y embeddings (Unidad 3)
    Comparar resultados.

### POS Y NER

La utilización de reglas fijas en el código para la búsqueda de palabras claves en los prompts, suelen quitar flexibilidad al sistema. Se valora la utilización de técnicas de análisis de texto flexibles, de tipo POS, NER, Lematización, o ingeniería de prompts,para evitar palabras “hardcoded” en la aplicación.

## Scrapear para la base de datos de grafos

In [3]:
!pip install requests beautifulsoup4

In [4]:
#Importo las librerías
import requests
from bs4 import BeautifulSoup


In [ ]:
# URL de ejemplo
url = "https://example.com"

# Hacer una solicitud HTTP GET a la URL
response = requests.get(url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    # Analizar el contenido HTML de la respuesta
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraer y mostrar el título de la página
    title = soup.title.string
    print(f"Título de la página: {title}")

    # Extraer todos los enlaces de la página
    links = soup.find_all('a')
    for link in links:
        print(link.get('href'))
else:
    print(f"Error: {response.status_code}")

In [15]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

## Implementación de BERT

In [16]:
import torch
from transformers import BertTokenizer, BertModel
from torch.nn.functional import cosine_similarity

# Cargar el tokenizador y el modelo BERT multilingüe
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Texto de entrada en español
text = "¡Hola, embeddings de BERT en idioma español son interesantes!"

# Tokenizar el texto y obtener los IDs de los tokens
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

# Convertir los IDs de los tokens a tensores y obtener los embeddings
token_ids = torch.tensor([token_ids])
with torch.no_grad():
    outputs = model(token_ids)
    embeddings = outputs.last_hidden_state

# Calcular la similitud del coseno entre dos palabras (por ejemplo, "idioma" y "español")
word1_idx = tokens.index("idioma")
word2_idx = tokens.index("español")
similarity = cosine_similarity(embeddings[0][word1_idx].unsqueeze(0), embeddings[0][word2_idx].unsqueeze(0))

print(f"Tokens: {tokens}")
print(f"Similitud entre 'idioma' y 'español': {similarity.item()}")

# Tokens: ['¡', 'Ho', '##la', ',', 'em', '##bed', '##ding', '##s', 'de', 'BE', '##RT', 'en', 'idioma', 'español', 'son', 'interesante', '##s', '!']
# Similitud entre 'idioma' y 'español': 0.6664961576461792

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Tokens: ['¡', 'Ho', '##la', ',', 'em', '##bed', '##ding', '##s', 'de', 'BE', '##RT', 'en', 'idioma', 'español', 'son', 'interesante', '##s', '!']
Similitud entre 'idioma' y 'español': 0.6664962768554688


In [26]:
context = """Personalidad
Al inicio de la serie, Rei Ayanami es presentada como una joven enigmática de carácter introvertido, silenciosa y nada comunicativa, que a pesar de confiar en las personas adultas, sólo manifiesta emociones y lealtad sincera hacia Gendo Ikari, con quien también mantiene una relación distante.

Rei vive en una calle desolada de Tokio-3, en el solitario apartamento número 402. Este apartamento demuestra que Rei lleva una vida monótona y aislada.

Además, resulta ser una representación de la habitación subterránea en la que nació y se crió, según dice Ritsuko Akagi en el episodio 23.

La posesión material más preciada de Rei son las gafas rotas de Gendo Ikari, que recogió del suelo cuando a éste se le cayeron al liberarla del Entry Plug eyectado del EVA-00.

En el episodio 05, Shinji va al apartamento de Rei para entregarle una identificación. Cuando ella sale de la ducha, parece no importarle que Shinji puede verla desnuda. En cambio, ella se molesta al ver que el joven sujeta las gafas de Gendo.

Cuando Shinji se cae encima de ella, Rei lo trata como una molestia menor y le pide que por favor se mueva. Ella luego se viste en su presencia, sin demostrar vergüenza o malestar alguno.

Más tarde, cuando Shinji intenta pedir disculpas a Rei por lo ocurrido, a ella no parece importarle. Esto demuestra que Rei posee tan poca experiencia relacionándose con otras personas, que no tiene ni idea del efecto que podría tener su desnudez sobre un adolescente.


"""

In [27]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# Cargar el modelo preentrenado de BERT y el tokenizador
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Definir el contexto y la pregunta
#context = "Gendo Ikari es el padre de Shinji Ikari en el anime Neon Genesis Evangelion."
question = "¿Qué es Rei Ayanami de Shinji Ikari?"

# Tokenizar la pregunta y el contexto
inputs = tokenizer(question, context, return_tensors='pt')

# Hacer la predicción
with torch.no_grad():
    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

# Obtener el índice de inicio y fin de la respuesta
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores) + 1

# Decodificar el resultado
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())
answer = ' '.join(tokens[answer_start:answer_end])

print(f"Respuesta: {answer}")



Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Respuesta: present ##ada como una jo ##ven enigma ##tica de cara ##cter intro ##vert ##ido


In [28]:
!pip install googlesearch-python

In [29]:
from googlesearch import search

# Término de búsqueda
query = "Padre de Shinji Ikari"

# Realizar la búsqueda y obtener los resultados
search_results = search(query, num_results=10, lang='es')

# Mostrar los resultados
for i, result in enumerate(search_results, start=1):
    print(f"Resultado {i}: {result}")


Resultado 1: https://es.wikipedia.org/wiki/Shinji_Ikari
Resultado 2: https://wiki.evageeks.org/es:Shinji_Ikari
Resultado 3: https://boo.world/es/database/profile/69131/gendo-ikari-personality-type
Resultado 4: https://es.wikipedia.org/wiki/Shinji_Ikari
Resultado 5: https://wiki.evageeks.org/es:Shinji_Ikari
Resultado 6: https://boo.world/es/database/profile/69131/gendo-ikari-personality-type
Resultado 7: https://es.wikipedia.org/wiki/Shinji_Ikari
Resultado 8: https://boo.world/es/database/profile/69131/gendo-ikari-personality-type
Resultado 9: https://boo.world/es/database/profile/69131/gendo-ikari-personality-type
Resultado 10: https://es.wikipedia.org/wiki/Shinji_Ikari
Resultado 11: https://boo.world/es/database/profile/69131/gendo-ikari-personality-type


In [20]:
answer

'padre'

## Word to vec skip gram

Nota: descargar el modelo tarda alrededor de 40 minutos :]

In [ ]:
!pip install gensim
!wget https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz

--2024-06-26 00:42:34--  https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz
Resolving cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)... 200.16.17.55
Connecting to cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)|200.16.17.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1123304474 (1.0G) [application/x-gzip]
Saving to: ‘SBW-vectors-300-min5.bin.gz’

       SBW-vectors-  90%[=================>  ] 969.80M   139KB/s    eta 4m 20s 

In [ ]:
from gensim.models import KeyedVectors

# Carga un modelo Word2Vec preentrenado (asegúrate de tener el archivo en tu directorio)
model = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.bin.gz', binary=True)

In [ ]:
# Información del modelo
print(model)

# Similitud entre dos palabras específicas
print(f"Similitud entre 2 palabras: {model.similarity('perro', 'conejo')}")

# Palabra de consulta
query_word = "gato"

# Encuentra las palabras más similares a la palabra de consulta
most_similar_words = model.most_similar(positive=[query_word], topn=10)

# Imprime las palabras más similares y sus similitudes de coseno
print(f'Palabras cercanas a {query_word}:')
for word, similarity in most_similar_words:
    print(f"Palabra: {word}, Similitud: {similarity}")

## Base de datos de grafos

Neo4j Aura

Username: neo4j
password: xujYU4dL5jknttByetMU_-fSDKRPhYcJiFWLrcl1vK4

In [1]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 1.8 MB/s eta 0:00:00


In [2]:
from neo4j import GraphDatabase

# Configura la conexión a Neo4j AuraDB
uri = "neo4j+s://8cc7e7ea.databases.neo4j.io"
user = "neo4j"
password = "xujYU4dL5jknttByetMU_-fSDKRPhYcJiFWLrcl1vK4"

# Crear la conexión
driver = GraphDatabase.driver(uri, auth=(user, password))

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]

# Ejemplo: Crear nodos
query = """
CREATE (a:Persona {nombre: 'Monolongo', edad: 30, ciudad: 'Madrid'}),
       (b:Persona {nombre: 'Bob2', edad: 24, ciudad: 'Barcelona'})
"""
run_query(query)

# Ejemplo: Consultar nodos
query = "MATCH (p:Persona) RETURN p.nombre, p.edad, p.ciudad"
results = run_query(query)

for record in results:
    print(record)


<Record p.nombre='Alice' p.edad=30 p.ciudad='Madrid'>
<Record p.nombre='Bob' p.edad=24 p.ciudad='Barcelona'>
<Record p.nombre='Monolongo' p.edad=30 p.ciudad='Madrid'>
<Record p.nombre='Bob2' p.edad=24 p.ciudad='Barcelona'>
